# Introduction

Using this Python notebook you will:
1. Understand 3 Chicago datasets  
1. Load the 3 datasets into 3 tables in a Db2 database
1. Execute SQL queries to answer assignment questions 

## Understand the datasets 
To complete the assignment problems in this notebook you will be using three datasets that are available on the city of Chicago's Data Portal:
1. <a href="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2">Socioeconomic Indicators in Chicago</a>
1. <a href="https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t">Chicago Public Schools</a>
1. <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2">Chicago Crime Data</a>

### 1. Socioeconomic Indicators in Chicago
This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

For this assignment you will use a snapshot of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2



### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

For this assignment you will use a snapshot of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/0g7kbanvn5l2gt2qu38ukooatnjqyuys.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t




### 3. Chicago Crime Data 

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. 

This dataset is quite large - over 1.5GB in size with over 6.5 million rows. For the purposes of this assignment we will use a much smaller sample of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2




### Problem 1

##### How many rows are in each dataset?

In [1]:
from ibm_db import connect
import pandas as pd
import ibm_db as id_
import ibm_db_dbi as idd

dsn_hostname = "dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net"  
dsn_uid      = "lwv08529"
dsn_pwd      = "m+t5tm80txtghqlw"
dsn_driver   = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_port     = "50000"
dsn_protocol = "TCPIP" 

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ib-d_.conn_errormsg() )

Connected to database:  BLUDB as user:  lwv08529 on host:  dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net


In [2]:
tables = "SELECT TABSCHEMA, TABNAME, CREATE_TIME FROM syscat.tables WHERE tabschema = 'LWV08529'"
tabs = id_.exec_immediate(conn, tables)
for i in range(11):
    print(id_.fetch_both(tabs))

{'TABSCHEMA': 'LWV08529', 0: 'LWV08529', 'TABNAME': 'EMPLOYEES', 1: 'EMPLOYEES', 'CREATE_TIME': datetime.datetime(2018, 12, 18, 2, 29, 45, 863431), 2: datetime.datetime(2018, 12, 18, 2, 29, 45, 863431)}
{'TABSCHEMA': 'LWV08529', 0: 'LWV08529', 'TABNAME': 'JOB_HISTORY', 1: 'JOB_HISTORY', 'CREATE_TIME': datetime.datetime(2018, 12, 18, 2, 29, 46, 6289), 2: datetime.datetime(2018, 12, 18, 2, 29, 46, 6289)}
{'TABSCHEMA': 'LWV08529', 0: 'LWV08529', 'TABNAME': 'JOBS', 1: 'JOBS', 'CREATE_TIME': datetime.datetime(2018, 12, 18, 2, 29, 46, 143428), 2: datetime.datetime(2018, 12, 18, 2, 29, 46, 143428)}
{'TABSCHEMA': 'LWV08529', 0: 'LWV08529', 'TABNAME': 'DEPARTMENTS', 1: 'DEPARTMENTS', 'CREATE_TIME': datetime.datetime(2018, 12, 18, 2, 29, 46, 287490), 2: datetime.datetime(2018, 12, 18, 2, 29, 46, 287490)}
{'TABSCHEMA': 'LWV08529', 0: 'LWV08529', 'TABNAME': 'LOCATIONS', 1: 'LOCATIONS', 'CREATE_TIME': datetime.datetime(2018, 12, 18, 2, 29, 46, 433561), 2: datetime.datetime(2018, 12, 18, 2, 29, 46, 

In [3]:
# Rows in Census Data (Socieconimic Indicators)
rowcensus = "SELECT COUNT(*) FROM SOCIO"
rs = id_.exec_immediate(conn, rowcensus)
print(id_.fetch_both(rs))

{'1': '78', 0: '78'}


In [4]:
# Rows in Public Schools
rowpublic = "SELECT COUNT(*) FROM REPORT_CARDS"
rr = id_.exec_immediate(conn, rowpublic)
print(id_.fetch_both(rr))

{'1': '566', 0: '566'}


In [5]:
# Rows in Crime Data
rowcrime = "SELECT COUNT(*) FROM CRIME"
rc = id_.exec_immediate(conn, rowcrime)
print(id_.fetch_both(rc))

{'1': '533', 0: '533'}


### Problem 2

##### Find average college enrollments by community area

(When taking a screenshot for sharing, the first 5-10 rows of the result set are sufficient)

In [6]:
avgcommunity = """ SELECT COMMUNITY_AREA_NAME, AVG(COLLEGE_ENROLLMENT) FROM REPORT_CARDS GROUP BY COMMUNITY_AREA_NAME LIMIT 10"""

ac = id_.exec_immediate(conn, avgcommunity)

for i in range(10):
    print(id_.fetch_both(ac))

{'COMMUNITY_AREA_NAME': 'ALBANY PARK', 0: 'ALBANY PARK', '2': '858.000000', 1: '858.000000'}
{'COMMUNITY_AREA_NAME': 'ARCHER HEIGHTS', 0: 'ARCHER HEIGHTS', '2': '2411.500000', 1: '2411.500000'}
{'COMMUNITY_AREA_NAME': 'ARMOUR SQUARE', 0: 'ARMOUR SQUARE', '2': '486.000000', 1: '486.000000'}
{'COMMUNITY_AREA_NAME': 'ASHBURN', 0: 'ASHBURN', '2': '810.375000', 1: '810.375000'}
{'COMMUNITY_AREA_NAME': 'AUBURN GRESHAM', 0: 'AUBURN GRESHAM', '2': '417.500000', 1: '417.500000'}
{'COMMUNITY_AREA_NAME': 'AUSTIN', 0: 'AUSTIN', '2': '475.347826', 1: '475.347826'}
{'COMMUNITY_AREA_NAME': 'AVALON PARK', 0: 'AVALON PARK', '2': '507.333333', 1: '507.333333'}
{'COMMUNITY_AREA_NAME': 'AVONDALE', 0: 'AVONDALE', '2': '910.000000', 1: '910.000000'}
{'COMMUNITY_AREA_NAME': 'BELMONT CRAGIN', 0: 'BELMONT CRAGIN', '2': '1198.833333', 1: '1198.833333'}
{'COMMUNITY_AREA_NAME': 'BEVERLY', 0: 'BEVERLY', '2': '409.000000', 1: '409.000000'}


### Problem 3

##### Find the number of schools that are healthy school certified

In [7]:
health = """SELECT DISTINCT(HEALTHY_SCHOOL_CERTIFIED) FROM REPORT_CARDS"""

hq = id_.exec_immediate(conn, health)
for i in range(2):
    print(id_.fetch_both(hq))
    
healthque = """SELECT COUNT(*) FROM REPORT_CARDS WHERE HEALTHY_SCHOOL_CERTIFIED = 'Yes'"""

hqs = id_.exec_immediate(conn, healthque)
print(id_.fetch_both(hqs)['1'])

{'HEALTHY_SCHOOL_CERTIFIED': 'No', 0: 'No'}
{'HEALTHY_SCHOOL_CERTIFIED': 'Yes', 0: 'Yes'}
16


### Problem 4

##### How many observations have a Location Description value of GAS STATION


In [8]:
locque = """SELECT DISTINCT("Location") FROM REPORT_CARDS """
lqs = id_.exec_immediate(conn, locque)
#print(id_.fetch_both(lqs))


locque = """SELECT COUNT(*) FROM CRIME WHERE LOCATION_DESCRIPTION = 'GAS STATION'"""

lq = id_.exec_immediate(conn, locque)

print(id_.fetch_both(lq)[0])

6


### Problem 5

##### Retrieve a list of the top 10 community areas which have most number of schools and sorted in descending order.

In [9]:
sc = """SELECT COMMUNITY_AREA_NAME, COUNT(NAME_OF_SCHOOL) FROM REPORT_CARDS GROUP BY COMMUNITY_AREA_NAME ORDER BY COUNT(NAME_OF_SCHOOL) DESC LIMIT 10"""

scq = id_.exec_immediate(conn, sc)

for i in range(10):
    print(id_.fetch_both(scq))

{'COMMUNITY_AREA_NAME': 'AUSTIN', 0: 'AUSTIN', '2': '23', 1: '23'}
{'COMMUNITY_AREA_NAME': 'SOUTH LAWNDALE', 0: 'SOUTH LAWNDALE', '2': '22', 1: '22'}
{'COMMUNITY_AREA_NAME': 'WEST TOWN', 0: 'WEST TOWN', '2': '20', 1: '20'}
{'COMMUNITY_AREA_NAME': 'ENGLEWOOD', 0: 'ENGLEWOOD', '2': '17', 1: '17'}
{'COMMUNITY_AREA_NAME': 'NEAR WEST SIDE', 0: 'NEAR WEST SIDE', '2': '16', 1: '16'}
{'COMMUNITY_AREA_NAME': 'NORTH LAWNDALE', 0: 'NORTH LAWNDALE', '2': '16', 1: '16'}
{'COMMUNITY_AREA_NAME': 'EAST GARFIELD PARK', 0: 'EAST GARFIELD PARK', '2': '13', 1: '13'}
{'COMMUNITY_AREA_NAME': 'ROSELAND', 0: 'ROSELAND', '2': '13', 1: '13'}
{'COMMUNITY_AREA_NAME': 'NEW CITY', 0: 'NEW CITY', '2': '13', 1: '13'}
{'COMMUNITY_AREA_NAME': 'HUMBOLDT PARK', 0: 'HUMBOLDT PARK', '2': '13', 1: '13'}


### Problem 6

##### How many observations have value MOTOR VEHICLE THEFT in the Primary Type variable (this is the number of crimes related to Motor vehicles)

In [10]:
l = """SELECT COUNT(*) FROM CRIME WHERE PRIMARY_TYPE = 'MOTOR VEHICLE THEFT' """

f = id_.exec_immediate(conn, l)

print(id_.fetch_both(f)[0])

24


### Problem 7

##### Use INNER JOIN to find the __MINIMUM__ (i.e. lowest) “Average Student Attendance” for the community area where hardship is 96. 

In [11]:
r = """SELECT MIN(rs.AVERAGE_STUDENT_ATTENDANCE) FROM REPORT_CARDS rs INNER JOIN SOCIO s ON rs.COMMUNITY_AREA_NUMBER = s.COMMUNITY_AREA_NUMBER WHERE s.HARDSHIP_INDEX = 96"""

rq = id_.exec_immediate(conn, r)

print(id_.fetch_both(rq))

{'1': '86.10%', 0: '86.10%'}
